# Imports

In [103]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [104]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [105]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# Get the Data

In [106]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 3.96 ms, sys: 1.89 ms, total: 5.85 ms
Wall time: 5.02 ms


In [107]:
y_train_all.shape

(60000,)

In [108]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [109]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 1.56 s, sys: 201 ms, total: 1.76 s
Wall time: 1.76 s


In [110]:
%%time

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 263 ms, sys: 40.6 ms, total: 303 ms
Wall time: 302 ms


# Explore

In [111]:
x_train_all.shape

(60000, 784)

In [112]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [113]:
y_train_all.shape

(60000,)

In [114]:
x_test.shape

(10000, 784)

In [115]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [116]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

### Convert target values to one-hot encoding

In [117]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [118]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [119]:
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [120]:
values

array([5, 0, 4, 1, 9])

In [121]:
values[4]

9

In [122]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [123]:
y_train_all.shape

(60000, 10)

In [124]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

## Create validation dataset from training data

In [125]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [126]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [127]:
x_train.shape

(50000, 784)

In [128]:
x_val.shape

(10000, 784)

In [129]:
class LogImages(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, x_data):
        super(LogImages, self).__init__()
        self.log_dir = log_dir
        self.x_data = x_data 
        
    def on_epoch_end(self, epoch, logs=None):
        # Log a batch of images at the end of an epoch
        file_writer = tf.summary.create_file_writer(self.log_dir + '/images')

        with file_writer.as_default():
            images = np.reshape(self.x_data[:4], (-1, 28, 28, 1))  # Log 4 images
            tf.summary.image("4 training data examples", images, max_outputs=4, step=epoch)

# Load the training Image

In [130]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

img_path = '/Users/kenny/Documents/DS Projects/MathGarden/MNIST/test_img.png'

img = load_img(img_path, color_mode='grayscale', target_size=(28, 28))

img_array = img_to_array(img) / 255.0

test_img = img_array.flatten()


# Setup Tensorflow Graph

In [131]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Neural network architecture parameters
n_hidden1 = 512  
n_hidden2 = 64   
NR_CLASSES = 10  
nr_epochs = 10   

# Define the Model
model = Sequential([
    Dense(n_hidden1, activation='relu', input_shape=(TOTAL_INPUTS,), name='layer_1'),  # Note the input_shape adjustment
    Dropout(0.2), 
    Dense(n_hidden2, activation='relu', name='layer_2'),
    Dense(NR_CLASSES, activation='softmax', name='output_layer')
])


# Compile the Model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Display model's architecture
model.summary()

# Train the Model
history = model.fit(x_train, y_train, epochs=nr_epochs, batch_size=1000, validation_data=(x_val, y_val))

# Evaluate the Model on the test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

# Model Predictions
predictions = model.predict(np.array([test_img])) 
predicted_class = np.argmax(predictions, axis=1)
print(f"Prediction for test image is {predicted_class}")


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 layer_2 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 10)                650       
                                                                 
Total params: 435402 (1.66 MB)
Trainable params: 435402 (1.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
50/50 [==============================] - 3s 27ms/step - loss: 0.6423 - accuracy: 0.8168 - val_loss: 0.2535 - val_accuracy: 0.9264
Epoch 2/

## Neural Network Architecture

In [132]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [133]:
model = Sequential([
    Dense(n_hidden1, activation='relu', input_shape=(TOTAL_INPUTS,), name='layer_1'),
    Dropout(0.2),
    Dense(n_hidden2, activation='relu', name='layer_2'),
    Dense(NR_CLASSES, activation='softmax', name='output_layer')
])

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


# Tensorboard Setup

In [134]:
%load_ext tensorboard
%tensorboard --logdir logs/fit



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 19019), started 0:31:06 ago. (Use '!kill 19019' to kill it.)

# Loss, Optimisation & Metrics

In [135]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Constants for the model
n_hidden1 = 512  
n_hidden2 = 64   
NR_CLASSES = 10  
TOTAL_INPUTS = 784 

# Define the Model
model = Sequential([
    Dense(n_hidden1, activation='relu', input_shape=(TOTAL_INPUTS,), name='layer_1'),
    Dropout(0.2), 
    Dense(n_hidden2, activation='relu', name='layer_2'),
    Dense(NR_CLASSES, activation='softmax', name='output_layer')
])

# Compile the Model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Display model's architecture
model.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 512)               401920    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 layer_2 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 10)                650       
                                                                 
Total params: 435402 (1.66 MB)
Trainable params: 435402 (1.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Run Session

In [136]:
# Define the batch size for training
size_of_batch = 1000 

history = model.fit(x_train, y_train,
                    epochs=nr_epochs,
                    batch_size=size_of_batch,
                    validation_data=(x_val, y_val))

print(history.history)

# Evaluate the Model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


Epoch 1/50
50/50 [==============================] - 2s 24ms/step - loss: 0.6179 - accuracy: 0.8235 - val_loss: 0.2599 - val_accuracy: 0.9265
Epoch 2/50
50/50 [==============================] - 1s 19ms/step - loss: 0.2316 - accuracy: 0.9321 - val_loss: 0.1856 - val_accuracy: 0.9462
Epoch 3/50
50/50 [==============================] - 1s 19ms/step - loss: 0.1677 - accuracy: 0.9510 - val_loss: 0.1503 - val_accuracy: 0.9566
Epoch 4/50
50/50 [==============================] - 1s 20ms/step - loss: 0.1330 - accuracy: 0.9615 - val_loss: 0.1275 - val_accuracy: 0.9623
Epoch 5/50
50/50 [==============================] - 1s 18ms/step - loss: 0.1068 - accuracy: 0.9687 - val_loss: 0.1101 - val_accuracy: 0.9681
Epoch 6/50
50/50 [==============================] - 1s 19ms/step - loss: 0.0914 - accuracy: 0.9727 - val_loss: 0.0983 - val_accuracy: 0.9708
Epoch 7/50
50/50 [==============================] - 1s 18ms/step - loss: 0.0749 - accuracy: 0.9785 - val_loss: 0.0947 - val_accuracy: 0.9711
Epoch 8/50
50

313/313 [==============================] - 1s 2ms/step - loss: 0.0737 - accuracy: 0.9821
Test Loss: 0.07373380661010742, Test Accuracy: 0.9821000099182129


# Make a Prediction

In [137]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load the image, ensuring it is in grayscale, and resize it to 28x28 pixels
img = load_img('MNIST/test_img.png', color_mode='grayscale', target_size=(28, 28))

# Convert the image to an array, normalize it, and possibly invert if required
img_array = img_to_array(img) / 255.0

# If your model was trained on inverted images, invert the pixel values
# img_array = 1.0 - img_array

# Flatten the array if your model expects flattened input
test_img = img_array.flatten()

# Make the prediction
predictions = model.predict(np.array([test_img]))
predicted_class = np.argmax(predictions, axis=1)
print(f'Prediction for test image is {predicted_class}')


1/1 [==============================] - 0s 70ms/step
Prediction for test image is [5]


In [138]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Accuracy on test set is {test_accuracy:.2%}')


313/313 [==============================] - 1s 2ms/step - loss: 0.0737 - accuracy: 0.9821
Accuracy on test set is 98.21%
